In [ ]:
import whisper
import numpy as np
import sounddevice as sd
import queue
import threading
import time

model = whisper.load_model("base")  # puoi usare "small", "medium", "large" se vuoi più accuratezza

q = queue.Queue()

# Parametri audio
samplerate = 16000  # Whisper usa 16kHz
blocksize = 4000  # circa 0.25 secondi
record_seconds = 5  # finestra mobile di 5 secondi

# Buffer per la registrazione
audio_buffer = np.zeros((samplerate * record_seconds,), dtype=np.float32)

def audio_callback(indata, frames, time_info, status):
    q.put(indata.copy())

def transcribe_live():
    global audio_buffer
    while True:
        try:
            data = q.get()
            audio_buffer = np.roll(audio_buffer, -len(data))
            audio_buffer[-len(data):] = data

            # trascrivi ogni secondo circa
            if time.time() % 1 < 0.1:
                audio_np = audio_buffer.copy()
                result = model.transcribe(audio_np, language='it', fp16=False)
                print("Testo:", result["text"])

        except Exception as e:
            print("Errore nella trascrizione:", e)

# Thread per la trascrizione
thread = threading.Thread(target=transcribe_live, daemon=True)
thread.start()

# Avvia la registrazione
with sd.InputStream(channels=1, samplerate=samplerate, callback=audio_callback, blocksize=blocksize):
    print("Inizia a parlare... (premi Ctrl+C per uscire)")
    try:
        while True:
            time.sleep(0.1)
    except KeyboardInterrupt:
        print("Interrotto")


In [ ]:
import whisper
from transformers import pipeline

# Carica il modello (puoi scegliere: tiny, base, small, medium, large)
model = whisper.load_model("base")

# Percorso del tuo file audio (può essere mp3, wav, m4a, ecc.)
audio_path = "Nuova registrazione.m4a"  # Sostituisci con il tuo file

# Trascrizione
result = model.transcribe(audio_path)

# Stampa il testo trascritto
print("Testo trascritto:")
print(result["text"])



#prova usando un modello NER per estrarre le entità rilevanti non è stato usato in quant mistral è capace da solo di fare NER
ner_pipeline = pipeline("ner", model="d4data/biomedical-ner-all", aggregation_strategy="simple")
entities = ner_pipeline(result["text"])

# 3. Visualizza le entità estratte
print("Testo trascritto:\n", result["text"])
print("\nEntità rilevanti:")
structured_data = {}

for entity in entities:
    if "pressure" in entity["word"].lower():
        structured_data["pressione_arteriosa"] = entity["word"]
    elif "temperature" in entity["word"].lower():
        structured_data["temperatura"] = entity["word"]
    # aggiungi altre regole
for entity in entities:
    print(entity)

print("Dati da inserire nella cartella clinica:")
print(structured_data)


/home/jovyan/.conda/envs/big_data/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jovyan/.conda/envs/big_data/lib/python3.10/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializatio

Testo trascritto:
 Allora questa è una prova, so cercando di usare Whisper prova 1 2 3 4


Device set to use cuda:0


Testo trascritto:
 Good morning, doctor. Over the past three days, I've had a high fever, with temperatures reaching up to 38.7 degrees Celsius. I've also been feeling very fatigued and have experienced widespread muscle aches, especially in my legs and back.Last night, I measured my blood pressure and it was 145 over 95. Normally, it's around 120 over 80.My resting heart rate is 95 beats per minute. I haven't had a cough, but I do have nasal congestion and a bit of a headache.I took two 1000 mg tablets of paracetamol yesterday and one this morning. I'm allergic to penicillin.The last time I had blood tests, about a month ago, my blood glucose level was 110 mg/dL, which the doctor said was slightly above normal.

Entità rilevanti:
{'entity_group': 'Duration', 'score': np.float32(0.9461581), 'word': 'over the past three days', 'start': 22, 'end': 46}
{'entity_group': 'Severity', 'score': np.float32(0.9870543), 'word': 'high', 'start': 59, 'end': 63}
{'entity_group': 'Diagnostic_procedur

In [3]:
import torch

# Verifica se CUDA (NVIDIA GPU) è disponibile
print(f"PyTorch CUDA disponibile: {torch.cuda.is_available()}")

# Se sì, mostra dettagli GPU
if torch.cuda.is_available():
    print(f"Numero di GPU: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memoria totale: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")
else:
    print("Nessuna GPU NVIDIA rilevata per PyTorch!")

PyTorch CUDA disponibile: True
Numero di GPU: 4
GPU 0: NVIDIA A100-SXM4-80GB
  Memoria totale: 79.15 GB
GPU 1: NVIDIA A100-SXM4-80GB
  Memoria totale: 79.15 GB
GPU 2: NVIDIA A100-SXM4-80GB
  Memoria totale: 79.15 GB
GPU 3: NVIDIA A100-SXM4-80GB
  Memoria totale: 79.15 GB


In [ ]:
import requests
import json
import os
import json
import re

def query_ollama(prompt, model='mistral'):
    url = 'http://127.0.0.1:11434/api/generate'
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        result = response.json()
        return result.get("response", "").strip()
    else:
        raise Exception(f"Errore {response.status_code}: {response.text}")


In [ ]:


# ---------------------
# 🔍 Estrazione entità ner tramite mistral per la compilazione della cartella clinica
# ---------------------

prompt = """
Leggi il seguente testo clinico e restituisci un dizionario JSON con i seguenti campi:

  "data": "",
  "ora_chiamata": "",
  "ora_partenza_ambulanza": "",
  "ora_arrivo_sul_posto": "",
  "ora_partenza_dal_posto": "",
  "ora_arrivo_ps": "",
  "ora_rientro_sede": "",
  "codice_uscita": "",
  "codice_rientro": "",
  "tipo_mezzo": "",
  "numero_intervento": "",
  "indirizzo_intervento": "",
  "via": "",
  "numero_civico": "",
  "citta": "",
  "provincia": "",
  "cap": "",
  "telefono_chiamante": "",
  "cognome_nome_paziente": "",
  "sesso": "",
  "data_nascita": "",
  "eta": "",
  "luogo_nascita": "",
  "provincia_nascita": "",
  "residenza": "",
  "indirizzo_residenza": "",
  "citta_residenza": "",
  "provincia_residenza": "",
  "codice_fiscale": "",
  "nome_chiamante": "",
  "cognome_chiamante": "",
  "relazione_con_paziente": "",
  "stato_coscienza": "",
  "sintomi": [],
  "dolore_localizzato": "",
  "allergie": "",
  "patologie_pregresse": "",
  "farmaci_assunti": "",
  "parametri_vitali": [
    {
      "tempo_rilevazione": "",
      "frequenza_cardiaca": "",
      "pressione_arteriosa": "",
      "frequenza_respiratoria": "",
      "saturazione_ossigeno": "",
      "glicemia": "",
      "temperatura": "",
      "AVPU": "",
      "pupille_PEARL": "",
      "ECG": ""
    }
  ],
  "ossigenoterapia": "",
  "flusso_ossigeno": "",
  "farmaci_somministrati": [],
  "accesso_venoso": "",
  "soluzione_endovenosa": "",
  "immobilizzazioni": "",
  "monitoraggio_continuo": "",
  "mezzo_trasporto": "",
  "posizionamento_paziente": "",
  "ospedale_destinazione": "",
  "reparto_destinazione": "",
  "paziente_rifiuto_trasporto": false,
  "forze_dell'ordine_presenti": None,
  "decesso_sul_posto": false,
  "paziente_consegnato_ps": true,
  "condizioni_consegna": "",
  "nome_soccorritore_1": "",
  "ruolo_soccorritore_1": "",
  "nome_soccorritore_2": "",
  "ruolo_soccorritore_2": "",
  "nome_soccorritore_3": "",
  "ruolo_soccorritore_3": "",
  "firma_responsabile": "",
  "note_intervento": "",
  "problemi_riscontrati": ""
}


}

Se un’informazione non è presente, inserisci `null` o una stringa vuota. Non aggiungere commenti o testo fuori dal JSON. Ecco il testo:



Testo clinico:

"""
prompt += result["text"]
risposta = query_ollama(prompt, model="mistral-large")  
print("Risposta:", risposta)

# Crea la cartella se non esiste
folder = "cartelle_cliniche"
os.makedirs(folder, exist_ok=True)

import os
import json
import re

def extract_json_blocks_from_code_blocks(text):
    """Estrae tutti i blocchi JSON dal testo"""
    pattern = r'```json\n(.*?)```'
    matches = re.findall(pattern, text, re.DOTALL)
    json_blocks = []
    for match in matches:
        try:
            json_blocks.append(json.loads(match.strip()))
        except json.JSONDecodeError as e:
            print(f"Errore nel parsing JSON: {e}")
    return json_blocks

def get_next_file_index(folder_path):
    """Calcola il prossimo indice disponibile per i file nella cartella"""
    if not os.path.exists(folder_path):
        return 1
    
    existing_files = [f for f in os.listdir(folder_path) if f.startswith('json_') and f.endswith('.json')]
    if not existing_files:
        return 1
    
    indices = []
    for f in existing_files:
        try:
            indices.append(int(f.split('_')[1].split('.')[0]))
        except (IndexError, ValueError):
            continue
    
    return max(indices) + 1 if indices else 1


folder_name = "cartelle_cliniche"
folder = os.path.join("./", folder_name)

# Crea la cartella se non esiste
os.makedirs(folder, exist_ok=True)

# Ottieni il prossimo indice disponibile
start_index = get_next_file_index(folder)

# Estrai e salva i JSON
json_blocks = extract_json_blocks_from_code_blocks(risposta)
for i, data in enumerate(json_blocks):
    file_index = start_index + i
    filename = os.path.join(folder, f"json_{file_index}.json")
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    print(f"Salvato: {filename}")

print(f"Tutti i file sono stati salvati in: {folder}")
print(f"File creati: {len(json_blocks)}")
print(f"Prossimo indice disponibile: {start_index + len(json_blocks)}")




Risposta: ```json
{
  "data": "",
  "ora_chiamata": "11:08",
  "ora_partenza_ambulanza": null,
  "ora_arrivo_sul_posto": null,
  "ora_partenza_dal_posto": null,
  "ora_arrivo_ps": null,
  "ora_rientro_sede": null,
  "codice_uscita": "D2",
  "codice_rientro": "D4",
  "tipo_mezzo": "ambulanza",
  "numero_intervento": "PR125",
  "indirizzo_intervento": "via Verdi 10, Parma",
  "via": "via Verdi",
  "numero_civico": "10",
  "citta": "Parma",
  "provincia": "",
  "cap": "",
  "telefono_chiamante": "0521‑000002",
  "cognome_nome_paziente": null,
  "sesso": "maschile",
  "data_nascita": null,
  "eta": "circa 35 anni",
  "luogo_nascita": "",
  "provincia_nascita": "",
  "residenza": null,
  "indirizzo_residenza": null,
  "citta_residenza": null,
  "provincia_residenza": null,
  "codice_fiscale": null,
  "nome_chiamante": "",
  "cognome_chiamante": "",
  "relazione_con_paziente": "",
  "stato_coscienza": "vigile e orientato",
  "sintomi": [
    "forti dolori addominali",
    "nausea intensa",
 

In [6]:
latex=r"""
\documentclass[a4paper]{article}

% ========================
% PACCHETTI
% ========================
\usepackage[italian]{babel}
\usepackage[utf8]{inputenc}
\usepackage{array}
\usepackage{tabularx}
\usepackage[table]{xcolor}
\usepackage{amssymb}
\usepackage{geometry}

% Margini pagina
\geometry{a4paper, left=2cm, right=2cm, top=2.5cm, bottom=2.5cm}

% ========================
% COMANDI PERSONALIZZATI
% ========================
\newcommand{\field}[1]{\underline{\makebox[#1]{\hspace*{\fill}}}} % Campo compilabile a lunghezza fissa
\newcommand{\checkbox}{$\square$~}                                % Casella da spuntare
\newcommand{\graycell}[1]{\cellcolor[gray]{0.85}\textbf{#1}}      % Intestazione grigia in grassetto
\newcolumntype{L}[1]{>{\raggedright\arraybackslash}p{#1}}         % Colonna allineata a sinistra con larghezza personalizzata

% ========================
% INIZIO DOCUMENTO
% ========================
\begin{document}

% ----------------------------------------
% INTESTAZIONE
% ----------------------------------------
\begin{center}
    {\LARGE \textbf{CROCE ROSSA ITALIANA}}\\
    \textbf{Comitato Provinciale di Venezia}
\end{center}

\vspace{0.5cm}

% ----------------------------------------
% SEZIONE CHIAMATA
% ----------------------------------------
\begin{tabular}{|L{0.5\textwidth}|L{0.5\textwidth}|}
\hline
\multicolumn{2}{|c|}{\graycell{Chiamata}} \\ \hline
\graycell{Data} & \\ \hline
\graycell{Luogo Intervento} & \\ \hline
\graycell{Ora chiamata} & \\ \hline
\graycell{Ora partenza} & \\ \hline
\graycell{Ora sul posto} & \\ \hline
\graycell{Condizione riferita} & \\ \hline
\graycell{Ora partenza posto} & \\ \hline
\graycell{Ora in PS} & \\ \hline
\graycell{Ora libero e operativo} & \\ \hline
\graycell{Recapito telefonico} & \\ \hline
\end{tabular}

\vspace{0.5cm}

% ----------------------------------------
% SEZIONE AMBULANZA
% ----------------------------------------
\begin{tabular}{|L{0.5\textwidth}|L{0.5\textwidth}|}
\hline
\multicolumn{2}{|c|}{\graycell{Ambulanza}} \\ \hline
\graycell{CRI} & \\ \hline
\graycell{Sezione} & \\ \hline
\end{tabular}

\vspace{0.5cm}

% ----------------------------------------
% SEZIONE EQUIPAGGIO
% ----------------------------------------
\begin{tabular}{|L{\textwidth}|}
\hline
\graycell{Equipaggio} \\ \hline
Autista: \\
Soccorritore 1: \\
Soccorritore 2: \\
Soccorritore 3: \\
Infermiere (IP): \\
Medico: \\ \hline
\end{tabular}

\vspace{0.5cm}

% ----------------------------------------
% SEZIONE CODICI
% ----------------------------------------
\begin{tabular}{|L{0.5\textwidth}|L{0.5\textwidth}|}
\hline
\graycell{Codice uscita} & \checkbox B \checkbox V \checkbox G \checkbox R \\ \hline
\graycell{Codice rientro} & \checkbox 0 \checkbox 1 \checkbox 2 \checkbox 3 \checkbox 4 \\ \hline
\end{tabular}

\vspace{0.5cm}

% ----------------------------------------
% SEZIONE PAZIENTE
% ----------------------------------------
\begin{tabular}{|L{0.25\textwidth}|L{0.25\textwidth}|L{0.25\textwidth}|L{0.25\textwidth}|}
\hline
\graycell{Cognome Nome} & & \graycell{Sesso} & \checkbox M \checkbox F \\ \hline
\graycell{Data nascita} & & \graycell{Luogo nascita} & \\ \hline
\end{tabular}

\vspace{0.5cm}

% ----------------------------------------
% SEZIONE PARAMETRI VITALI
% ----------------------------------------
\begin{tabular}{|L{2cm}|L{3cm}|L{3cm}|L{3cm}|}
\hline
 & \graycell{T1 (orario)} & \graycell{T2 (orario)} & \graycell{T3 (orario)} \\ \hline
FC (battiti/min) & & & \\ \hline
PA (pressione) & & & \\ \hline
SpO2 (\%) & & & \\ \hline
GCS & & & \\ \hline
\end{tabular}

\vspace{0.5cm}

% ----------------------------------------
% SEZIONE SINTOMI E INTERVENTO
% ----------------------------------------
\begin{tabular}{|L{\textwidth}|}
\hline
\graycell{Sintomi principali} \\ \hline
\checkbox Dolore toracico \checkbox Difficoltà respiratoria \checkbox Trauma \checkbox Altro: \\ \hline
\end{tabular}

\vspace{0.2cm}

\begin{tabular}{|L{0.5\textwidth}|L{0.5\textwidth}|}
\hline
\graycell{Trattamenti effettuati} & \graycell{Farmaci somministrati} \\ \hline
\checkbox Ossigeno \checkbox Immobilizzazione \checkbox Medicazione & Nessuno \\ \hline
\end{tabular}

\vspace{0.5cm}

% ----------------------------------------
% SEZIONE AUTORITÀ
% ----------------------------------------
\begin{tabular}{|L{\textwidth}|}
\hline
\graycell{Autorità presenti} \\ \hline
\checkbox Polizia \checkbox Carabinieri \checkbox Vigili del fuoco \checkbox Altri: \\ \hline
\end{tabular}

\vspace{0.5cm}

% ----------------------------------------
% SEZIONE ESITO
% ----------------------------------------
\begin{tabular}{|L{\textwidth}|}
\hline
\graycell{Esito} \\ \hline
\checkbox Trasporto in PS \checkbox Rifiuto trattamento \checkbox Decesso \\
Ospedale destinazione: Ospedale Civile di Venezia \\ \hline
\end{tabular}

\vspace{0.5cm}

% ----------------------------------------
% SEZIONE NOTE
% ----------------------------------------
\begin{tabular}{|L{\textwidth}|}
\hline
\graycell{Note} \\ \hline
\\ \hline
\end{tabular}

\vspace{0.5cm}

% ----------------------------------------
% SEZIONE FIRME
% ----------------------------------------
\begin{tabular}{|L{0.5\textwidth}|L{0.5\textwidth}|}
\hline
\graycell{Operatore} & \graycell{Data e firma} \\ \hline
 &  \\ \hline
\end{tabular}

\end{document}



"""
prompt_temaple="Compila il seguente template LaTeX "+latex+" con questi dati"+risposta+": Sostituisci ogni campo con il contenuto appropriato in base ai dati. Devi mantenere la struttura originale LaTeX se alcune informazioni non sono presenti lascia il campo vuoto e restituisci l’intero file `.tex`."
risposta_latex = query_ollama(prompt_temaple, model="Mistral-large")  # oppure "dolphin-mistral" o "fauno"
print("Risposta:", risposta_latex)


# Puoi anche provare a parsare in JSON:

try:
    dati = json.loads(risposta)
    print("JSON:", dati)
except json.JSONDecodeError:
    print("La risposta non è un JSON valido.")

Risposta: ```latex
\documentclass[a4paper]{article}

% ========================
% PACCHETTI
% ========================
\usepackage[italian]{babel}
\usepackage[utf8]{inputenc}
\usepackage{array}
\usepackage{tabularx}
\usepackage[table]{xcolor}
\usepackage{amssymb}
\usepackage{geometry}

% Margini pagina
\geometry{a4paper, left=2cm, right=2cm, top=2.5cm, bottom=2.5cm}

% ========================
% COMANDI PERSONALIZZATI
% ========================
\newcommand{\field}[1]{\underline{\makebox[#1]{\hspace*{\fill}}}} % Campo compilabile a lunghezza fissa
\newcommand{\checkbox}{$\square$~}                                % Casella da spuntare
\newcommand{\graycell}[1]{\cellcolor[gray]{0.85}\textbf{#1}}      % Intestazione grigia in grassetto
\newcolumntype{L}[1]{>{\raggedright\arraybackslash}p{#1}}         % Colonna allineata a sinistra con larghezza personalizzata

% ========================
% INIZIO DOCUMENTO
% ========================
\begin{document}

% --------------------------------------

In [3]:
#compilazione del codice latex e salvataggio in un file .tex e produzione pdflatex
import os
import re
import subprocess
import shutil

def estrai_latex(text):
    """
    Estrae il codice LaTeX da una stringa, anche se racchiuso tra blocchi markdown.
    """
    # Cerca blocco ```latex ... ```
    match = re.search(r"```latex(.*?)```", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()

def salva_e_compila_latex(risposta_latex, nome_file_base="cartella_clinica"):
    # Cartelle di output
    latex_dir = "latex_output"
    pdf_dir = "pdf_output"
    os.makedirs(latex_dir, exist_ok=True)
    os.makedirs(pdf_dir, exist_ok=True)

    # Estrai codice LaTeX puro
    codice_latex = estrai_latex(risposta_latex)

    # Percorsi file
    tex_path = os.path.join(latex_dir, f"{nome_file_base}.tex")
    pdf_path = os.path.join(pdf_dir, f"{nome_file_base}.pdf")

    # Salva il file .tex
    with open(tex_path, "w", encoding="utf-8") as f:
        f.write(codice_latex)
    print(f"File LaTeX salvato in: {tex_path}")

    # Compila con pdflatex nella cartella latex_dir
    command = ['pdflatex', '-interaction=nonstopmode', os.path.basename(tex_path)]
    process = subprocess.run(command, cwd=latex_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Controlla la compilazione e sposta il PDF
    if process.returncode == 0:
        print("Compilazione avvenuta con successo!")
        # Sposta il PDF nella cartella pdf_dir
        pdf_generato = os.path.join(latex_dir, f"{nome_file_base}.pdf")
        if os.path.exists(pdf_generato):
            shutil.move(pdf_generato, pdf_path)
            print(f"PDF salvato in: {pdf_path}")
        else:
            print("Attenzione: PDF non trovato dopo la compilazione.")
    else:
        print("Errore nella compilazione LaTeX:")
        print(process.stdout)
        print(process.stderr)

# Esempio d'uso:
salva_e_compila_latex(risposta_latex, nome_file_base="cartella_clinica_1")


File LaTeX salvato in: latex_output/cartella_clinica_1.tex
Errore nella compilazione LaTeX:
This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./cartella_clinica_1.tex
LaTeX2e <2023-11-01> patch level 1
L3 programming layer <2024-01-22>

! LaTeX Error: Missing \begin{document}.

See the LaTeX manual or LaTeX Companion for explanation.
Type  H <return>  for immediate help.
 ...                                              
                                                  
l.1 l
     atex
Missing character: There is no l in font nullfont!
Missing character: There is no a in font nullfont!
Missing character: There is no t in font nullfont!
Missing character: There is no e in font nullfont!
Missing character: There is no x in font nullfont!
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(/usr/share/texlive/t